# Evaluating Healthcare Plans on Healthcare.gov

Choosing from the dizzying array of health plans on Healthcare.gov can be a daunting task. Wouldn't it be great if there was a way to systematically evaluate the plan alternatives?

If you are like me, you are looking at the health plans on Healthcare.gov and scratching your head. While you likely have only a handful of providers to choose from each provider offers an array of plans. Compounding the problem is the reality that you have to pick a plan before you know what claims you’ll have for the year. Since you aren’t [Nostradamus]( https://en.wikipedia.org/wiki/Nostradamus), I’ve created this script to ease your selection anxiety and aid in the decision process, using the dead simple concept of expected value.

This script uses the HealthCare.gov [total cost estimate feature](https://www.healthcare.gov/choose-a-plan/your-total-costs/) and a little of your own intuition to calculate an expected value for each plan.  Since the expected value represent a cost outlay, the objective is to select the plan with the minimum expected value.


## More formally

\begin{equation*}
Expected Value_{plan} = \sum(TC_i P_i) \\
\end{equation*}

\begin{equation*}
Objective: \min(Expected Value_{plan})  \\
\end{equation*}

Where:
- TC<sub>i</sub>: Total Cost for each (ith) scenario (Low, Medium, or High), provided by Healthcare.gov, which includes monthly premiums, deductible and estimated out of pocket expenses. The scenarios vary with inputs such as age you'll have to review the scenarios on Healthcare.gov for your set of inputs.
- P<sub>i</sub>: Your estimated probability of occurence for each (ith) plan outcome (Low, Medium, High). Using a little intuition and reflection on your claims history you can assign a probability to each scenario. 
- Expected Value<sub>plan</sub>: A weighted average value for each plan, using the probabilities as weights. 

For example, lets assume the following scenarios are provided by Healthcare.gov for my inputs and example plan A:

- Low: \$1800, no claims or out of pocket costs (premium costs only)
- Medium: \$2100, two office visits and a couple prescriptions
- High: \$9000, one Hospitalization, 10 office visits and several prescriptions

Now reviewing each scenario I decide the probability of the Low, Medium and High outcomes are 10%, 85%, and 5% respectively (total probability must equal 1). 

Therefore the expected value for plan A is:

2415 = 1800 \* 0.10 + 2100 \* 0.85 + 9000 \* 0.05

The expected value in this case is closest to the medium cost outcome of \$2100 as it is assigned the highest probability, but is also pushed higher by the \$9000 high cost outcome.


### Note about Expected Value

For decisions involving uncertainty, the concept of [expected value](https://en.wikipedia.org/wiki/Expected_value) provides a rational way for selecting the best course of action. The calculated amount for each plan does not represent the amount you’ll actually pay, rather it is a means to aid in the decision making process by quantitatively evaluating each plan.  

It should not be the sole factor in your decisions process as it does not include many factors that are important in the decision such as the type of plan (EPO, PPO, HMO, etc.), provider availability, coverage in your area, while traveling, etc.). 

Also, this problem formulation assumes you are perfectly rational and therefore doesn't apply a [utility function](https://en.wikipedia.org/wiki/Utility). If you are risk adverse, for example, you may need to apply a separate utility function.

### Limitations (Disclaimer):
- This script has not been thoroughly tested, nor have all variable, locations, and use cases. I wrote it for myself then offered it as a potential aid to others. It worked for me but may not work for you. Results should be double checked. Script is provided as is, I make no warranties.
- The total cost feature on Healthcare.gov could be wrong, as of the time of this writing 12/12/15 it is listed as Beta, and states “This feature is new some information may be incorrect or incomplete”. You should double check the figures before making a final decision.
- The script makes certain assumptions such as you are selecting plans as an individual and may not work for families or with other inputs. 


## The Healthcare.gov Webscraper

The script works by sending an HTTP Post request to Healthcare.gov with the inputs you provide. It then iterates through the results collecting the total cost estimtes as well as some basic plan informaiton. Finally the expected value is calculated and lowest value plans are displayed. The user may optionally export the results to a csv file for further analysis.

### Your Inputs
Input the values into the widget below as appropriate, sample values are provided to guide you in the input format.

In [1]:
# Run this cell to display the input widgets
import ipywidgets as widgets
from IPython.display import display

high_prob = widgets.FloatSlider(
    value = 0.10,
    min = 0,
    max = 1,
    description='High Outcome:',
    step = 0.01,
)

med_prob = widgets.FloatSlider(
    value = 0.80,
    min = 0,
    max = 1,
    description='Medium Outcome:',
    step = 0.01,
)

low_prob = widgets.FloatSlider(
    value = 0.10,
    min = 0,
    max = 1,
    description='Low Outcome:',
    step = 0.01,
)


caption = widgets.HTML(value = '<b>Assign a probability value to each High, Medium, and Low \
outcome below. The total probability must sum to 1</b>')
container = widgets.Box(children=[high_prob, med_prob, low_prob])
display(caption, container)


married = widgets.Checkbox(
    description='Married?',
    value=False,
)

age = widgets.BoundedIntText(
    value=28,
    min=18,
    max=110,
    description='Age:',
)

gender = widgets.ToggleButtons(
    description='Gender:',
    options=['Male', 'Female'],
)

is_parent = widgets.Checkbox(
    description='Parent? (unsupported)',
    value=False,
    disabled = True,
)

is_pregnant = widgets.Checkbox(
    description='Pregnant?',
    value=False,
)

uses_tobacco = widgets.Checkbox(
    description='Use Tobacco?',
    value=False,
)

state = widgets.Dropdown(
    options = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI',\
             'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', \
             'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', \
             'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', \
             'WV', 'WI', 'WY'],
    value = 'SC',
    description ='State',
)

market = 'Individual'

zipcode = widgets.IntText(
    value=29001,
    description='Zip Code:',
)

year = widgets.BoundedIntText(
    value = 2016,
    min = 2016,
    max = 2100,
    description='Year of plan benefits:',
)

csv = widgets.Checkbox(
    description = 'Export to csv?',
    value = False,
)

caption2 = widgets.HTML(value = '<br> <b> Input the values into the widget below\
    sample values are provided to guide you in the input format.</b><br>')
container2 = widgets.Box(children=[caption2, age, gender, married, is_parent, is_pregnant, \
                                   uses_tobacco, state, zipcode, year, csv])
display(container2)

In [2]:
# Check inputs
assert high_prob.value + med_prob.value + low_prob.value == 1, AssertionError("Probabilities must sum to 1")

import requests

url_state = 'https://marketplace.api.healthcare.gov/api/v1/states/'
request_state = requests.get(url_state + state.value)
state_data = request_state.json()

assert len(state_data['shop_hix_url']) == 0, AssertionError('This app only works with ' +
    'Healthcare.gov, the state you provided offers a customized marketplace ' + 
    'for health plans at ' + str(state_data['shop_hix_url']))


print("inputs passed")

inputs passed


In [3]:
# Select County
url_zip = 'https://marketplace.api.healthcare.gov/api/v1/counties/by/zip/'

request_zip = requests.get(url_zip + str(zipcode.value))
county_data = request_zip.json()


if len(county_data['counties']) > 1:
    county_dict = {}
    #counties = [x['name'] for x in county_data['counties']]
    
    for county in county_data['counties']:
        county_dict[county['name']] = county
        
    
    
    county_name = widgets.ToggleButtons(
    description='Select your county:',
    options=list(county_dict.keys()),
    )
    
    display(county_name)
else:
    print("County selected: " +  county_data['counties'][0]['name'])


In [4]:
# Actual Script do not change values
import json
import pandas as pd

url = 'https://marketplace.api.healthcare.gov/api/v1/plans/search'
utilization_level = ["Low", "Medium", "High"]

limit = 10
offset = 0
payload = {"filter":{"division":"HealthCare"},
           "household": {"income":-1,"people":
            [{"age":age.value,"is_pregnant":is_pregnant.value,"is_parent":is_parent.value,
              "uses_tobacco":uses_tobacco.value,
              "gender":gender.value,"utilization_level":"Low"}],
            "has_married_couple":married.value},
           "market":market,"limit":limit,"offset":offset,"order":"asc",
           "place":{"countyfips":county_dict[county_name.value]['fips'],"state":state.value,
           "zipcode":str(zipcode.value)},"sort":"premium","year":year.value}


response = requests.post(url, data=json.dumps(payload))
fixtures = response.json()
result_total = fixtures['total']
print("Total Number of Plan Results", result_total)

plan_dict = {}

for level in utilization_level:
    payload['household']['people'][0]['utilization_level'] = level
    offset = 0
    remaining_results = result_total
    
    while remaining_results > 0:

        payload["offset"] = offset
        response = requests.post(url, data=json.dumps(payload))
        fixtures = response.json()
        
        if level == "Low":
            for plan in fixtures['plans']:
                plan_dict[plan['id']] = {'Name': plan['name'],\
                                         'Company': plan['issuer']['name'], \
                                         'Premium': plan['premium'], 'Type': plan['type'],\
                                         'Deductible': plan['moops'][0]['amount'],\
                                         'Low': plan['oopc'] + plan['premium'] * 12}
        else:
            for plan in fixtures['plans']:
                plan_dict[plan['id']][level] = plan['oopc'] + (plan['premium'] * 12)
                #plan_dict[plan['id']][level + ' oopc'] = plan['oopc']

        offset += limit
        remaining_results -= limit



# Convert to dataframe        
df = pd.DataFrame(plan_dict).transpose()

# Calc expected value
df['Expected_Value'] = df['High'] * high_prob.value + df["Medium"] * \
med_prob.value + df["Low"] * low_prob.value

df = df.sort("Expected_Value")


if csv.value:
    import os
    
    df.to_csv('Healthplans_expected_value_comparision.csv')
    print("Exported 'healthplans_expected_value_comparision.csv' to file path " + os.getcwd())

print("Lowest Expected Value Plan:")
display(df[:1])

print("All plans, sorted by Expected Value (asc):")
display(df)



Total Number of Plan Results 54
Lowest Expected Value Plan:


,Company,Deductible,High,Low,Medium,Name,Premium,Type,Expected_Value
49532SC0380004,BlueChoice HealthPlan,6850,9376.942,2519.348,2746.295,Blue Option Catastrophic,209.89,EPO,3386.665


All plans, sorted by Expected Value (asc):


,Company,Deductible,High,Low,Medium,Name,Premium,Type,Expected_Value
49532SC0380004,BlueChoice HealthPlan,6850,9376.942,2519.348,2746.295,Blue Option Catastrophic,209.89,EPO,3386.665
26065SC0390001,BlueCross BlueShield of South Carolina,6850,9444.992,2535.428,2821.177,BlueEssentials Catastrophic 1,211.23,EPO,3454.983
49532SC0380003,BlueChoice HealthPlan,6750,9606.102,2848.508,3093.955,Blue Option Bronze 6750,237.32,EPO,3720.625
49532SC0380035,BlueChoice HealthPlan,6850,9676.582,2818.988,3104.737,Blue Option Bronze 6500,234.86,EPO,3733.346
49532SC0380022,BlueChoice HealthPlan,6850,9728.782,2871.188,3101.835,Blue Option Bronze 6850,239.21,EPO,3741.465
26065SC0380006,BlueCross BlueShield of South Carolina,6850,9732.992,2823.428,3109.177,BlueEssentials HD Bronze 2,235.23,EPO,3742.983
26065SC0380014,BlueCross BlueShield of South Carolina,6550,9467.672,2858.108,3143.857,BlueEssentials HD Bronze 5,238.12,EPO,3747.663
49532SC0380021,BlueChoice HealthPlan,6250,9151.822,2894.228,3179.977,Blue Option Bronze 6250 HD,241.13,EPO,3748.586
49532SC0380020,BlueChoice HealthPlan,6850,9732.382,2874.788,3116.535,Blue Option Bronze 4500,239.51,EPO,3753.945
49532SC0380001,BlueChoice HealthPlan,6600,9476.142,2868.548,3154.297,Blue Option Bronze 5001 HD,238.99,EPO,3757.906


#### I hope this proved useful to you, let me know how it worked for you.